# How Authentication Tokens Are Created and Stored in this App

Looking at your codebase, the token creation and storage happen in the Login.tsx component. Here's the flow:

## 1. Token Creation (Server-side)

When a user logs in, the backend server generates a token (typically a JWT - JSON Web Token). This happens when you make a login request from your Login.tsx component:

```tsx
fetchLogin({
  url: "/login",
  method: "POST",
  schema: z.object({
    data: z.object({
      token: z.string(),
    }),
  }),
  axiosConfig: {
    data: {
      key,
      password,
    },
  },
});
```

## 2. Token Storage (Client-side)

After a successful login, your app stores the token in the encrypted localStorage using the `StorageHelper` class:

```tsx
useEffect(() => {
  if (loginData.data) {
    storageHelper.setItem("token", loginData.data.data.token);
    return navigate("/");
  }
}, [loginData.data]);
```

The `StorageHelper` class encrypts the token using AES encryption from CryptoJS before storing it:

```typescript
setItem(name: string, value: object | string): void {
  if (!this.storage) return;
  let dataToStore: string;
  if (value === null) {
    dataToStore = "null";
  } else if (typeof value === "object") {
    dataToStore = JSON.stringify(value);
  } else {
    dataToStore = value;
  }
  this.storage.setItem(name, this.encrypt(dataToStore));
}
```

## 3. Token Usage in API Requests

When making API calls, your `AxiosHelper` class automatically attaches the token to request headers:

```typescript
getAxios() {
  this.axios.interceptors.request.use(
    async (config) => {
      try {
        const token = this.storageHelper.getItem("token");
        if (token) {
          config.headers.Authorization = `Bearer ${token}`;
        }
        return config;
      } catch (error) {
        return Promise.reject(error);
      }
    },
    // error handling
  );
  // ...
}
```

## 4. Token Verification (AuthMiddleware)

The `AuthMiddleware` component checks if the token exists and is valid:

```tsx
useEffect(() => {
  if (stateUser.getUser()?.data) {
    setLoading(false);
    return;
  }
  const token = storageHelper.getItem("token");
  if (token) {
    fetchUser({
      url: "/user",
      method: "GET",
      schema: UserModel,
    });
    return;
  }
  setLoading(false);
  navigate("/auth/login");
}, []);
```

If a token is found, it makes a request to `/user` to verify the token is valid and fetch user data.

To implement your own login and token storage:

1. Make a login request to your authentication endpoint
2. When successful, store the returned token in localStorage using `StorageHelper`
3. Use the token in subsequent API requests
4. Ensure your `AuthMiddleware` checks for the token and verifies it by making an API request

This pattern ensures the token is securely stored and automatically used for authentication across your application.